In [ ]:
import numpy as np
from pprint import pprint

import random, pickle 
import pandas as pd

from sklearn.datasets import make_regression
# from autosklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split

import autosklearn
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.classification import AutoSklearnClassifier

In [ ]:
df = pd.read_pickle("data_latest_average_non_zero_raw_secondary_task_repeated_insole.pkl")

In [ ]:
df.columns

In [ ]:
participant_list = df.participant_number.drop_duplicates().to_list()
random.Random(420).shuffle(participant_list)
print(participant_list)

participant_list_train, participant_list_test = train_test_split(participant_list, random_state=115,  train_size=0.85)

print(participant_list_train, participant_list_test)

In [ ]:
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

df_test = df[df['participant_number'].isin(participant_list_test)] #select test participants
df_test = df_test.sample(frac=1).reset_index(drop=True) #shuffle and reset_index
# df_test.reset_index(inplace=True,drop=True)
df_train = df[df['participant_number'].isin(participant_list_train)] #select train participants
df_train = df_train.sample(frac=1).reset_index(drop=True) #shuffle and reset_index
# df_train.reset_index(inplace=True,drop=True)

In [ ]:
arr = df_train[['mfk_1_entlasten', 'mfk_2_entlasten',
       'mfk_3_entlasten', 'mfk_4_entlasten', 'mfk_5_entlasten',
       'zehe_1_entlasten', 'zehe_2_entlasten', 'zehe_3_entlasten',
       'zehe_4_entlasten', 'zehe_5_entlasten', 'pelotten_hoehe',
       'pelotten_form', 'laengsgewoelbe_hoehe', 'basis_5_entlasten',
       'aussenrand_anheben', 'innenrand_anheben']].values

arr_np = np.asarray(arr.tolist())
y_train = arr_np.astype(float)
y_train += 0.0000000000000000000000000000001

x_train = np.asarray(df_train.input_features.tolist())

In [ ]:
arr = df_test[['mfk_1_entlasten', 'mfk_2_entlasten',
       'mfk_3_entlasten', 'mfk_4_entlasten', 'mfk_5_entlasten',
       'zehe_1_entlasten', 'zehe_2_entlasten', 'zehe_3_entlasten',
       'zehe_4_entlasten', 'zehe_5_entlasten', 'pelotten_hoehe',
       'pelotten_form', 'laengsgewoelbe_hoehe', 'basis_5_entlasten',
       'aussenrand_anheben', 'innenrand_anheben']].values

arr_np = np.asarray(arr.tolist())
y_test = arr_np.astype(float)
y_test += 0.0000000000000000000000000000001

x_test = np.asarray(df_test.input_features.tolist())

In [ ]:
x_train.shape, y_train.shape , x_test.shape, y_test.shape

In [ ]:
automl = AutoSklearnRegressor(
    time_left_for_this_task=3600*5,
    per_run_time_limit=150*2,
    memory_limit = 500000,
    seed = 14141,
    metric = autosklearn.metrics.mean_squared_error,
    
    # resampling_strategy = 'cv'
    resampling_strategy_arguments = {
    "shuffle": True,        # Whether to shuffle before splitting data
    # "folds": 3              # Used in 'cv' based resampling strategies
    }
)
automl.fit(x_train, y_train)

In [ ]:
# save model
# with open('data_averaged_time5hours_perRun300_secondary_task.pkl', 'wb') as f:
#     pickle.dump(automl, f)

print(automl.leaderboard())

In [ ]:
pprint(automl.show_models(), indent=4)

In [ ]:
predictions = automl.predict(x_test)
print("Accuracy score:", accuracy_score(y_test, predictions))

In [ ]:
df_test['predictions'] = predictions.tolist()
df_test['y_test'] = y_test.tolist()

In [ ]:
# df_test.to_pickle("data_averaged_time5hours_perRun300_secondary_task.pkl")

In [ ]:
# df_test.to_csv("data_averaged_time5hours_perRun300_secondary_task.csv", index=False)

In [ ]:
predictions, y_test